In [ ]:
import sklearn
import tensorflow as tf



# Load The Data

# Collaborative Filtering Model

In [ ]:
w = tf.variable(3.0)
x = tf.

# Content-Based Filtering Model

In [ ]:
user_NN = 

# Training

# Testing